In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm_notebook

In [2]:
driver=webdriver.Chrome("c:/mychrome/chromedriver.exe")

In [3]:
url="https://map.kakao.com/"
driver.get(url)
time.sleep(5)

driver.find_element_by_css_selector('.layer_body').click()
search=driver.find_element_by_css_selector('#search\.keyword\.query')
search.send_keys("제주도맛집")
driver.find_element_by_id('search.keyword.submit').click()

In [4]:
# 가게명 카테고리 정보 추출하는 함수
def get_store_info(stores):
    store_info={'Name':[], 'Category':[]}
    for i, store in enumerate(stores):
        try:
            name=store.select("div > strong > a.link_name")[0].text.strip()
            category=store.select("div > span")[0].text.strip()
        except:
            name=""
            category=""
        
        # 딕셔너리 업데이트
        store_info['Name'].append(name)
        store_info['Category'].append(category)
    return store_info

In [5]:
# 가게 별점 & 평점 정보 추출하는 함수
def get_store_ratings(ratings):
    rating_info={'Star':[], 'Star_Count':[], 'Review_Count':[]}
    for i, rating in enumerate(ratings):
        try:
            star=rating.select("div > span > em")[0].text.strip()
            star_cnt=rating.select("div > span > a")[0].text.strip().replace('건','').replace(',','')
            review_cnt=rating.select("div > a > em")[0].text.strip().replace(',','')
        except:
            star=0
            star_cnt=0
            review_cnt=0
        
        # 딕셔너리 업데이트
        rating_info['Star'].append(float(star))
        rating_info['Star_Count'].append(int(star_cnt))
        rating_info['Review_Count'].append(int(review_cnt))
    return rating_info

In [6]:
# 가게 주소 & 영업 시간 & 전화 정보 추출하는 함수
def get_store_addr(addrs):
    addr_info={'Address':[], 'Hour':[], 'Phone':[]}
    for i, addr in enumerate(addrs):
        try:
            location=addr.select("div.addr > p")[0].text.strip()
            hour=addr.select("div.openhour > p > a")[0].text.strip()
            phone=addr.select("div.contact.clickArea > span.phone")[0].text.strip()
        except:
            location=""
            hour=""
            phone=""
        
        # 딕셔너리 업데이트
        addr_info['Address'].append(location)
        addr_info['Hour'].append(hour)
        addr_info['Phone'].append(phone)
    return addr_info

In [7]:
page=1
df_list=[]

while True:
    # 데이터 수집
    html=driver.page_source
    soup=BeautifulSoup(html, 'html.parser')
    page=page+1
    
    # 식당 정보 크롤링
    stores=soup.select("div > ul.placelist > li > div.head_item.clickArea")
    store_info=get_store_info(stores)
    
    ratings=soup.select("div > ul.placelist > li > div.rating.clickArea")
    rating_info=get_store_ratings(ratings)
    
    addrs=soup.select("div > ul.placelist > li > div.info_item")
    addr_info=get_store_addr(addrs)
    
    # 데이터프레임 만들기
    df1=pd.DataFrame(store_info)
    df2=pd.DataFrame(rating_info)
    df3=pd.DataFrame(addr_info)
    df_fin=pd.concat([df1, df2, df3], axis=1)
    df_list.append(df_fin)
    
    # 다음 페이지로 넘기기
    try:
        if page>21:
            break
        elif page==2:
            driver.find_element_by_xpath("//*[@id='info.search.place.more']").click()
            time.sleep(2)
        elif page%5==1:
            driver.find_element_by_xpath("//*[@id='info.search.page.next']").click()
            time.sleep(3)
        else:
            idx=page%5
            if idx==2:
                driver.find_element_by_xpath("//*[@id='info.search.page.no2']").click()
                time.sleep(2)
            elif idx==3:
                driver.find_element_by_xpath("//*[@id='info.search.page.no3']").click()
                time.sleep(1)
            elif idx==4:
                driver.find_element_by_xpath("//*[@id='info.search.page.no4']").click()
                time.sleep(3)
            else:
                driver.find_element_by_xpath("//*[@id='info.search.page.no5']").click()
                time.sleep(2)
    except:
        print(page)
        break

In [8]:
driver.close()

In [9]:
jeju=pd.concat(df_list).reset_index(drop=True)
jeju

,Name,Category,Star,Star_Count,Review_Count,Address,Hour,Phone
0,대원가,"굴,전복",3.6,150,72,제주특별자치도 제주시 도남로16길 15 재영샤르빌 101호,영업시간 매일 09:00 ~ 21:00,064-753-3030
1,이금돈지,"해물,생선",3.7,32,263,제주특별자치도 제주시 노연로 143,영업시간 매일 10:00 ~ 22:00,064-711-8866
2,돈향기,삼겹살,4.1,54,221,제주특별자치도 제주시 관덕로15길 30,영업시간 매일 11:30 ~ 23:30,064-724-0228
3,랜디스도넛 제주직영점,도넛,3.5,93,384,제주특별자치도 제주시 애월읍 애월로 27-1,영업시간 매일 10:00 ~ 20:00,064-799-0610
4,연돈,"돈까스,우동",4.1,253,195,제주특별자치도 서귀포시 일주서로 968-10 1층,,
...,...,...,...,...,...,...,...,...
313,하하호호 우도본점,햄버거,4.1,118,219,제주특별자치도 제주시 우도면 우도해안길 532,,010-2899-1365
314,르토아베이스먼트,디저트카페,3.8,40,239,제주특별자치도 제주시 구좌읍 해맞이해안로 1226,영업시간 매일 10:00 ~ 19:00,
315,골목카페옥수,카페,3.1,24,133,제주특별자치도 제주시 애월읍 소길1길 19,"영업시간 월,화,수,목,토,일 11:00 ~ 19:00",
316,싱싱잇,한식,3.1,24,89,제주특별자치도 제주시 한림읍 한림로 181,매일 17:00 ~ 04:00,


In [10]:
jeju.to_csv("kakaomap_jeju_crawling.csv", index=False, encoding='utf-8-sig')